In [1]:
#importing modules
import rospy
import time
import math
import jupyros as jr
import numpy as np
import assignment2_research_track_1.msg
import actionlib
import actionlib.msg
import matplotlib.pyplot as plt
import tf
import ipywidgets as widgets
from assignment2_research_track_1.srv import _node_B_goal
from geometry_msgs.msg import Pose, Point
from geometry_msgs.msg import PoseStamped
from assignment2_research_track_1.msg import PlanningAction
from nav_msgs.msg import Odometry
from IPython.display import display
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
from std_srvs.srv import *
import sys
import signal
import assignment2_research_track_1.msg

%matplotlib widget

In [2]:
rospy.init_node('Node_A_1') #starting Node_A_1
client = actionlib.SimpleActionClient('/reaching_goal',assignment_2_2022.msg.PlanningAction )#implementing action client
client.wait_for_server()

True

In [3]:
x = widgets.IntSlider(min=-100, max=100) #insert value for x coordinate
y = widgets.IntSlider(min=-100, max=100) #insert value for y coordinate
lsr =widgets.FloatText( description='Distance from closest obstacle:', disabled=True)#creating widget
out_lsr=widgets.Output() #output widget
x_pos=widgets.FloatText(description='Current robot x coordinate :', disabled=True) #creating widget
y_pos=widgets.FloatText(description='Current robot y coordinate :', disabled=True) #creating widget
check1 = widgets.Checkbox( #check-box for sending goal
value=False,
description='Send goal',
disabled=False
)
check2 = widgets.Checkbox( #check-box for cancelling goal
value=False,
description='Cancel goal',
disabled=False
)
confirm_action = widgets.Button(description="Confirm action") #button for confirming action
out = widgets.Output() #widget output
menu = widgets.VBox([check1, check2, confirm_action]) #vbox for widgets

In [4]:
global client, status

def on_confirm_clicked(confirm_action): #if button is clicked
    with out:
        if check1.value==True and check2.value==False:
            Position()
        if check2.value==True and check1.value==False:
            Cancel()
confirm_action.on_click(on_confirm_clicked)

def Position(): #sending coordinates to robot
    global goal_x, goal_y
    goal_x = float(x.value)
    goal_y = float(y.value)
    goal = assignment2_research_track_1.msg.PlanningGoal()
    status=client.get_state()
    if not (status==actionlib.GoalStatus.ACTIVE):
        goal.target_pose.pose.position.x = goal_x
        goal.target_pose.pose.position.y = goal_y
        client.send_goal(goal)
        print("Desired position sent")


def Cancel(): #cancelling movement of robot
    status = client.get_state()
    if status == actionlib.GoalStatus.ACTIVE:
        client.cancel_goal()
        print("Canceling movement of robot")
print("Enter x position:") #enter x position
display(x) #display slider
print("Enter y position:")#enter y position
display(y) #display slider
display(menu,out) #display widgets

Enter x position:


IntSlider(value=0, min=-100)

Enter y position:


IntSlider(value=0, min=-100)

Output()

In [5]:
global goal_x, goal_y, client, status #global variables
class Plot: #class Plot
    def __init__(self): #init function
        self.fig, (self.plot1, self.plot2) = plt.subplots(1,2, figsize=(10,4)) #creating subplots
        self.trajectory, = self.plot1.plot([], [], markersize='3', label='Trajectory') #creating plot
        self.plot1.plot(goal_x, goal_y, marker='x', color='blue', markersize='8', label='Current goal')#creating plot
        self.plot1.plot(goal_x, goal_y, marker='v', color='green', markersize='8', label='Reached goals')#creating plot
        self.plot1.plot(goal_x, goal_y, marker='o', color='red', markersize='8', label='Canceled goals')#creating plot
        self.x = [] #empty lists
        self.y = [] #empty lists
        self.reached_goal = 0 #counter for reached goals
        self.canceled_goal = 0 #counter for canceled goals
        self.if_reached=False #checking if goal is reached
        self.if_canceled=False #checking if goal is canceled
        
    def plot_initialize(self): #initializing plot
        self.plot1.set_xlim(-20, 20)
        self.plot1.set_ylim(-20, 20)
        self.plot2.set_ylim(0, 10)
        return self.trajectory
    
    def Publish(self, data): #publishing data to lists
        self.x.append(data.pose.pose.position.x)
        self.y.append(data.pose.pose.position.y)
        x_pos.value=data.pose.pose.position.x #getting current position on x axis
        y_pos.value=data.pose.pose.position.y #getting current position on y axis
    
    def update(self, frame): #updating plots
        self.trajectory.set_data(self.x, self.y)
        status=client.get_state()
        if status==3 and not(self.if_reached): #if goal is reached
            self.plot1.plot(goal_x, goal_y, marker='v', color='green', markersize='8', label='Reached goals')
            self.reached_goal += 1
            self.plot2.bar(['reached', 'canceled'], [self.reached_goal, self.canceled_goal], color = 'orange')
            self.if_reached=True
        elif status==2 and not(self.if_canceled): #if goal is canceled
            self.plot1.plot(goal_x, goal_y, marker='o', color='red', markersize='8', label='Canceled goals')
            self.canceled_goal += 1
            self.plot2.bar(['reached', 'canceled'], [self.reached_goal, self.canceled_goal], color = 'orange')
            self.if_canceled=True
        elif (not(status==2) and self.if_canceled): #if goal is canceled
            self.if_canceled=False
        elif (not(status==3) and self.if_reached): #if goal is reached
            self.if_reached=False 
        elif (not(status==3) and not(status==2)): #if robot is in movement
            self.plot1.plot(goal_x, goal_y, marker='x', color='blue', markersize='8', label='Current goal')    
        self.plot2.bar(['reached', 'canceled'], [self.reached_goal, self.canceled_goal], color = 'orange')
        return self.trajectory

In [6]:
vis = Plot()#creating object
vis.plot1.set_title('Robot trajectory')#setting title
vis.plot1.set_xlabel('X coordinate')#setting label
vis.plot1.set_ylabel('Y coordinate')#setting label
vis.plot1.legend(bbox_to_anchor=(1.0, 1.0), loc='lower left')#creating legend
plt.tight_layout() #adjusting subplots
vis.plot1.grid() #creating grid
vis.plot2.set_title('Goals') #setting title
vis.plot2.set_xlabel('Status of goal') #setting label
vis.plot2.set_ylabel('Amount') #setting label
subscrib = rospy.Subscriber('/odom', Odometry, vis.Publish) #subscribing to topic
time.sleep(0.5) #delay
ani = FuncAnimation(vis.fig, vis.update, init_func=vis.plot_initialize) #drawing function
plt.show(block=True) #showing plot


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
display(x_pos) #displaying current x position
display(y_pos) #displaying current x position

FloatText(value=1.2603452485296789, description='Current robot x coordinate :', disabled=True)

FloatText(value=1.708498461286743, description='Current robot y coordinate :', disabled=True)

In [8]:
global min_dist
def laser(scan): #Finding closest obstacle using laser
    min_dist = scan.range_max
    for i in scan.ranges: #selecting the closest obstacle using simple algorithm
        if i < min_dist:
            min_dist = i       
    lsr.value = min_dist
jr.subscribe('/scan', LaserScan, laser) #subscribing to topic
time.sleep(0.5) #delay
display(lsr, out_lsr) #displaying value

FloatText(value=2.0389363765716553, description='Distance from closest obstacle:', disabled=True)

Output()